In [84]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from collections import Counter
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

In [91]:
strokeDf = pd.read_csv("healthcare-dataset-stroke-data.csv")
strokeDf = strokeDf.drop(["id", "hypertension"], axis = 1)

strokeDf.head()

,gender,age,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [92]:
print(strokeDf.isnull().sum())

strokeDf = strokeDf.dropna()

gender                 0
age                    0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64


In [93]:
strokeDf.isnull().values.any()

False

OneHot Encoded data

In [101]:
oneHotDf = pd.get_dummies(strokeDf, columns=['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'])

oneHotDf.head()

,age,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,1,228.69,36.6,1,0,1,0,0,1,...,0,1,0,0,0,1,0,1,0,0
2,80.0,1,105.92,32.5,1,0,1,0,0,1,...,0,1,0,0,1,0,0,0,1,0
3,49.0,0,171.23,34.4,1,1,0,0,0,1,...,0,1,0,0,0,1,0,0,0,1
4,79.0,0,174.12,24.0,1,1,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
5,81.0,0,186.21,29.0,1,0,1,0,0,1,...,0,1,0,0,0,1,0,1,0,0


In [111]:
xOneHot = oneHotDf.drop(['stroke'], axis = 1)
yOneHot = oneHotDf['stroke']



In [112]:
oversample = SMOTE()
xOneHot, yOneHot = oversample.fit_resample(xOneHot, yOneHot)
print(Counter(yOneHot))

Counter({1: 4700, 0: 4700})


In [113]:
xOneHot_train, xOneHot_test, yOneHot_train, yOneHot_test = train_test_split(xOneHot, yOneHot, random_state = 42, stratify = yOneHot, train_size = 0.75)
counter = Counter(yOneHot_train)
print(counter)

Counter({0: 3525, 1: 3525})


In [114]:
scaler=StandardScaler()
con_cols=['age','avg_glucose_level','bmi']
xOneHot_train[con_cols]=scaler.fit_transform(xOneHot_train[con_cols])
xOneHot_test[con_cols]=scaler.transform(xOneHot_test[con_cols])
xOneHot_train.head()

,age,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
2731,0.082898,0,-0.513744,0.616158,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0
4621,-1.728580,0,-0.711679,-0.791479,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0
5019,0.351987,1,2.265485,0.938313,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
3761,-1.049275,0,-0.321967,2.111772,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0
8870,0.831100,0,1.829646,0.858290,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0


In [115]:
classifier_oneHotKNN = KNeighborsClassifier()

classifier_oneHotKNN.fit(xOneHot_train, yOneHot_train)

oneHotKNN = classifier_oneHotKNN.predict(xOneHot_test)

print(confusion_matrix(yOneHot_test, oneHotKNN))
print('accuracy score :',accuracy_score(oneHotKNN,yOneHot_test))
print('Precision Score :',precision_score(oneHotKNN, yOneHot_test, average='weighted'))


[[1135   40]
 [  29 1146]]
accuracy score : 0.9706382978723405
Precision Score : 0.970682118605704
